In [1]:
from array import array
import math

# Keplerian Motion

The great physicist, Johannes Kepler, discovered that the planets orbit the Sun in elliptical orbits.  The Sun is located at one of the foci of the ellipse.  The eccentricity of the ellipse is given by the formula:

$$e = \sqrt{1-\frac{b^2}{a^2}}$$

where $a$ is the semi-major axis and $b$ is the semi-minor axis.  The distance from the centre of the ellipse to the focus is given by the formula:

$$f = \sqrt{a^2-b^2}$$

The orbit of the Earth around the Sun is almost circular with an eccentricity of $e = 0.0167$.  The orbit of Halley's Comet is highly elliptical with an eccentricity of $e = 0.967$.

The equations of motion for a particle in an elliptical orbit are given by:

$$\frac{d^2x}{dt^2} = -\frac{GMx}{r^3}$$
$$\frac{d^2y}{dt^2} = -\frac{GMy}{r^3}$$

where $r = \sqrt{x^2+y^2}$ and $G$ is the gravitational constant.  The initial conditions for the position and velocity of the particle are given by:

$$x(0) = a+f$$
$$y(0) = 0$$

$$v_x(0) = 0$$
$$v_y(0) = \sqrt{\frac{GM}{a}}$$

where $a$ is the semi-major axis of the ellipse.

# Circular Orbit

The simplest case of an elliptical orbit is a circular orbit.  In this case, the semi-major axis is equal to the semi-minor axis, so the eccentricity is zero.  The equations of motion for a circular orbit are given by:

$$\frac{d^2x}{dt^2} = -\frac{GMx}{r^3}$$
$$\frac{d^2y}{dt^2} = -\frac{GMy}{r^3}$$

where $r = \sqrt{x^2+y^2}$ and $G$ is the gravitational constant.  The initial conditions for the position and velocity of the particle are given by:

$$x(0) = r_0$$
$$y(0) = 0$$
$$v_x(0) = 0$$
$$v_y(0) = \sqrt{\frac{GM}{r_0}}$$

where $r_0$ is the radius of the circular orbit.

For planetary motion, a common system of units is one where distances are measured in astronomical units (1 A.U. = 
$1.496 \times 10^{11}$ m) and time is measured in years (1 year = $3.156 \times 10^7$ s).  In SI units, the gravitational constant is given by:

$$G = 1.32712442099 \times 10^{20} \frac{m^3}{s^2}$$

The mass of the Sun is given by:

$$M = 1.989 \times 10^{30} kg$$

Converting to astronomical units and years, the gravitational constant times the mass of the Sun is given by:

$$GM = 1.32712442099 \times 10^{20} \frac{m^3}{s^2} \times \frac{1}{(1.496 \times 10^{11} m)^3} \times \frac{(3.156 \times 10^7 s)^2}{1 year} = 4 \pi^2$$

Assuming the earth's orbit to be circular, the radius of the earth's orbit is given by:

$$r_0 = 1 A.U. = 1.496 \times 10^{11} m$$

and the initial velocity will be:

$$v_0 = \sqrt{\frac{GM}{r_0}} = \sqrt{\frac{4 \pi^2}{1}} = 2 \pi$$

In [2]:
# Circular Orbit of radius 1 for testing
#
#r0 = 1.0
#v0 = 2.0*math.pi
#
# Earth - circular with zero eccentricity
#
semi_major = 1.0
eccentricity = 0.0
#
# Halley's Comet
#
#semi_major = 17.834
#eccentricity = 0.967

nStep = 1000 # number of steps to calculate (i.e 10 years for a timestep of 0.01 years)
tau = 0.01 # time step (in years)
method = 3 # 1 = Euler, 2 = Euler-Cromer, 3 = Runge-Kutta, 4 = Adaptive Runge-Kutta

# Everything below this point is calculated.
semi_minor = semi_major*math.sqrt(1-eccentricity*eccentricity)
GMSIUnits = 1.32712442099E+20
SecondsInYear = 31556952
AUnit = 149597870700
GM = GMSIUnits/math.pow(AUnit,3)*math.pow(SecondsInYear,2)
F = math.sqrt((semi_major)**2-(semi_minor)**2)  # distance from centre of the ellipse to focus
r0 = semi_major+F

v0 = math.sqrt(GM*(2/r0-1/semi_major))
print("Initial Conditions:")
print("Semi-major axis = ", semi_major, "A.U.")
print("Semi-minor axis = ", semi_minor, "A.U.")
print("Focus = ", F, "A.U.")
print("GM = ", GM, "A.U.^3/year^2")
print("Initial radius = ", r0, "A.U.")
print("Initial velocity = ", v0, "A.U./year")

param = array('d')
param.append(GM)
mass = 1.0
adaptErr = 0.001
time = 0.0

Initial Conditions:
Semi-major axis =  1.0 A.U.
Semi-minor axis =  1.0 A.U.
Focus =  0.0 A.U.
GM =  39.47530582087944 A.U.^3/year^2
Initial radius =  1.0 A.U.
Initial velocity =  6.282937674438561 A.U./year


In [3]:
# Set up the arrays that will hold the position, velocity, and acceleration components
# The state vector will contain the x and y components of the position and velocity

r = array('d') # two components
v = array('d') # two components
state = array('d') # four components
accel = array('d')  # two components
deriv = array('d')  # four components

nState = 4

r.append(r0)
r.append(0.0)
v.append(0.0)
v.append(v0)
accel.append(0.0)
accel.append(0.0)

state.append(r[0])
state.append(r[1])
state.append(v[0])
state.append(v[1])

for i in range(0,nState):
    deriv.append(0.0)
    
rplot = array('d') # plot of radius vs. theta
thplot = array('d') # plot of theta vs. time
tplot = array('d') # plot of radius vs. time
kinetic = array('d') # plot of kinetic energy vs. time
potential = array('d') # plot of potential energy vs. time
energy = array('d') 

# Runge-Kutta Methods

We see that with a relatively small timestep, and using the Euler-Cromer method (which should be more stable due
to its reversibility property) that we still have (numerical) oscillations in quantities that should be constant.  Thus, we
seek an improved algorithm.  One such approach is the Runge-Kutta method(s).

Runge-Kutta methods work as follows:

Let an initial value problem be defined as follows:

$$\frac{dy}{dt} = f(t,y)$$
$$y(t_0) = y_0$$

We calculate the value of the function at subsequent timesteps according to:

$$y_{n+1} = y_n + \frac{\tau}{6}(k_1 + 2*k_2 + 2*k_3 + k_4)$$
$$t_{n+1} = t_n + \tau$$

where:

$$k_1 = f(t_n, y_n)$$
$$k_2 = f(t_n+\frac{\tau}{2}, y_n + \tau\frac{k_1}{2})$$
$$k_3 = f(t_n+\frac{\tau}{2}, y_n + \tau\frac{k_2}{2})$$
$$k_4 = f(t_n + \tau, y_n +\tau k_3)$$

So, in order to implement this in code, we need two things:

1. A function which calculates $f(t,y)$, which is the RHS of the differential equation, for various values of $t$ and $y$.
2. A function which calculates $y_{n+1}$ and $t_{n+1}$, using the $f(t,y)$ function defined in Step 1.

These two functions are defined in rk_functions.py, as gravrk() and rk4().  Note that in our case, we are actually solving FOUR differential equations at once (two for position and two for velocity), as defined above.


In [6]:
import rk_functions as rk

for iStep in range(0,nStep):
    normR = math.sqrt(r[0]*r[0]+r[1]*r[1])
    normV = math.sqrt(v[0]*v[0]+v[1]*v[1])
    rplot.append(normR)
    thplot.append(math.atan2(r[1],r[0]))
    tplot.append(time)
    kinetic.append(0.5*mass*normV*normV)
    potential.append(-1.0*GM*mass/normR)
    energy.append(kinetic[iStep]+potential[iStep])
    
    if (method == 1):
        accel[0] = -1.0*GM*r[0]/(normR*normR*normR)
        accel[1] = -1.0*GM*r[1]/(normR*normR*normR)
        r[0] = r[0] + tau*v[0]
        r[1] = r[1] + tau*v[1]
        v[0] = v[0] + tau*accel[0]
        v[1] = v[1] + tau*accel[1]
        print(r[0],r[1],v[0],v[1])
        time = time + tau
    else:
        if (method == 2):
            accel[0] = -1.0*GM*r[0]/(normR*normR*normR)
            accel[1] = -1.0*GM*r[1]/(normR*normR*normR)
            v[0] = v[0] + tau*accel[0]
            v[1] = v[1] + tau*accel[1]
            r[0] = r[0] + tau*v[0]
            r[1] = r[1] + tau*v[1]
            #print(r[0],r[1],v[0],v[1])
            time = time + tau
        else:
            if (method == 3):
                rk.rk4(state,nState,time,tau,deriv,param)
                r[0]=state[0]
                r[1]=state[1]
                v[0]=state[2]
                v[1]=state[3]
                time = time + tau
            else:
                if (method == 4):
                    rk.rka(state,nState,time,tau,adaptErr,deriv,param)
                    r[0]=state[0]
                    r[1]=state[1]
                    v[0]=state[2]
                    v[1]=state[3]
                    time = time + tau

TypeError: 'module' object is not callable

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8, 10))

# Keplerian Motion - Radius vs. Theta
ax1.plot(thplot, rplot, marker='', linestyle='-', color='blue')
ax1.set_title('Keplerian Motion')
ax1.set_xlabel('Theta')
ax1.set_ylabel('Radius')
ax1.set_ylim(0, 4.0)
ax1.grid(True)

# Keplerian Motion - Radius vs. Time
ax2.plot(tplot, rplot, marker='', linestyle='-', color='blue')
ax2.set_title('Keplerian Motion')
ax2.set_xlabel('Time')
ax2.set_ylabel('Radius')
ax2.set_ylim(0, 4.0)
ax2.grid(True)

# Keplerian Motion - Time vs. Energy
ax3.plot(tplot, energy, marker='', linestyle='-', color='red', label='Energy')
ax3.plot(tplot, potential, marker='', linestyle='-', color='green', label='Potential Energy')
ax3.plot(tplot, kinetic, marker='', linestyle='-', color='blue', label='Kinetic Energy')
ax3.set_title('Keplerian Motion')
ax3.set_xlabel('Time')
ax3.set_ylabel('Energy')
ax3.set_ylim(-40, 40)
ax3.legend()
ax3.grid(True)

plt.tight_layout()
plt.show()